## Importing Libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo

## Creating Class of WebCrawler

In [100]:
class Web_crawler_IMDB:

    def __init__(self):
        print('Web Crawler Class Created !!!','\n==============================')
    
    #Writing a function that retrives links of next (given number x) pages (Includes given link) 
    def url_get(self,links,x):
        self.url=[links]
        self.x=x
        for i in range(self.x):
            url_req_data = requests.get(self.url[-1])
            url_movie_soup = BeautifulSoup(url_req_data.content, 'html.parser')
            url_body = url_movie_soup.find_all('div', {'class': 'desc'})[-1]
            url_page='https://www.imdb.com'+url_body.find('a',{'class':'lister-page-next next-page'}).get('href')+'&ref_=adv_prv'
            self.url.append(url_page)
            print('Fetching URLs...')
        #URL=self.url
        
        print('------------------------------',"\nURLs Extracted !!!",'\n------------------------------')
        return self.url




    def extract_data(self,link):
        self.url=link
        #A Function that extracts data like Title, Time Period, Rating, Genre, Duration, Votes, Directors, Stars, Description etc.
        #These Movie data is extracted from IMDB site
        req_data = requests.get(self.url)     #Requesting data of given link
        review_soup = BeautifulSoup(req_data.content, 'html.parser')     #Extracting html content of a page
        body = review_soup.find_all('div', {'class': 'lister-item mode-advanced'})

        #Fetching and storing data in MongoDB
        try:
            dbConn = pymongo.MongoClient("mongodb://localhost:27017/")  # opening a connection to MongoDB
            db = dbConn['IMDB']
            table = db['IMDB_data']
            filename = 'IMDB_data'+".csv"
            fw = open(filename, "a")

            #Looping over every listed Movie on a perticular page
            for movie in body:


                #Getting the Title
                title_class=movie.find('h3',{'class':'lister-item-header'})
                title=title_class.find('a').text



                #Getting the time-period
                try:
                    time_period=title_class.find('span',{'class':'lister-item-year text-muted unbold'}).text.strip()
                except:
                    time_period="Not Found"



                #Getting the genre
                try:
                    genre=movie.find('span',{'class':'genre'}).text.rstrip().replace('\n','')
                except:
                    genre="Not Found"



                #Getting Duration of movie
                try:
                    duration=movie.find('span',{'class':'runtime'}).text[:3]
                except:
                    duration="Not Found"



                #Getting Directors and Stars    
                people_inv=movie.find('p',{'class':""}).text.strip().replace('\n','').split('|')
                people_inv=[people.strip() for people in people_inv]
                people_inv=[people[people.find(':')+1:] for people in people_inv]
                if len(people_inv)==2:
                    directors=people_inv[0]
                    stars=people_inv[1]
                else:
                    stars=people_inv[0]
                    directors="Not Found"



                #Getting the Rating
                try:

                    rating=movie.find('strong').text
                except:
                    rating="Not Found"



                #Getting Votes
                try:
                    vot=movie.find('p',{'class':'sort-num_votes-visible'})
                    votes=vot.find_all('span')[1].text
                except:
                    votes="Not Found"



                #Getting Description
                try:
                    description=movie.find_all("p", class_="text-muted")[-1].text.lstrip()
                except:
                    description='Not Found'



                #Writing in the file
                fw.write(title+","+time_period + "," + rating + "," + genre.replace(',',':') + ',' + duration.replace(',',':') + "," + votes.replace(',',':')+','+directors.replace(",",":")+','+stars.replace(',',':')+','+description.replace(',',':')+"\n")


                #Creating dictionary for storing it in MongoDB
                mydict={"Title":title, "Time Period":time_period, "Rating":rating, "Genre":genre, "Duration":duration, "Votes":votes, "Directors":directors, "Stars":stars, "Description":description}    


                #Inserting into Dataset in MongoDB
                table.insert_one(mydict)
                

            #Closing the file
            fw.close()
            

        #For checking is anything wrong with try block
        except:
            print('Something is wrong')
            
    #Function for extracting data of any no of links        
    def extract_pages(self,link):
        self.URL=link
        for i in range(len(self.URL)):
            self.extract_data(self.URL[i])
            print("Extracted Data of Page no.:",i)
        print('------------------------------','\nExtraction Completed !!!','\n------------------------------')


## Creating a new File

In [101]:
#Creating a File
filename = 'IMDB_data'+".csv"
fw = open(filename, "w")
headers = "Title, Time Period, Rating, Genre, Duration, Votes, Directors, Stars, Description \n"
fw.write(headers)
fw.close()

## Extracting Data

In [102]:
#Giving a link 
link='https://www.imdb.com/search/title/?title_type=feature,tv_series&count=200&ref_=adv_prv'
no_of_next_pages=300

#Creating a object from Class Web_crawler_IMDB
abc=Web_crawler_IMDB()

#Fetching URLs for next pages
URL=abc.url_get(link,no_of_next_pages)

#Extracting data for given URLS
abc.extract_pages(URL)

Web Crawler Class Created !!! 
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...
Fetching URLs...


Extracted Data of Page no.: 96
Extracted Data of Page no.: 97
Extracted Data of Page no.: 98
Extracted Data of Page no.: 99
Extracted Data of Page no.: 100
Extracted Data of Page no.: 101
Extracted Data of Page no.: 102
Extracted Data of Page no.: 103
Extracted Data of Page no.: 104
Extracted Data of Page no.: 105
Extracted Data of Page no.: 106
Extracted Data of Page no.: 107
Extracted Data of Page no.: 108
Extracted Data of Page no.: 109
Extracted Data of Page no.: 110
Extracted Data of Page no.: 111
Extracted Data of Page no.: 112
Extracted Data of Page no.: 113
Extracted Data of Page no.: 114
Extracted Data of Page no.: 115
Extracted Data of Page no.: 116
Extracted Data of Page no.: 117
Extracted Data of Page no.: 118
Extracted Data of Page no.: 119
Extracted Data of Page no.: 120
Extracted Data of Page no.: 121
Extracted Data of Page no.: 122
Extracted Data of Page no.: 123
Extracted Data of Page no.: 124
Extracted Data of Page no.: 125
Extracted Data of Page no.: 126
Extracted Da

## Fetching data from MongoDB and creating a .csv file

In [117]:
dbConn = pymongo.MongoClient("mongodb://localhost:27017/")  # opening a connection to MongoDB
db = dbConn['IMDB']
table = db['IMDB_data']

#Storing entire data into a list
data_mongo=[]
for x in table.find():
    data_mongo.append(x)

In [111]:
#Creating DataFrame from list
mongo_data=pd.DataFrame(data_mongo,columns=['Title','Time Period','Rating','Genre','Duration','Votes','Directors','Stars','Description'])

In [112]:
#Saving DataFrame in a .csv format
mongo_data.to_csv('Movies(60000).csv')

In [119]:
#Reading Data from .csv file
data=pd.read_csv('Movies(60000).csv',index_col=0)
data

,Title,Time Period,Rating,Genre,Duration,Votes,Directors,Stars,Description
0,Project Power,(2020),6.0,"Action, Crime, Sci-Fi",113,"25,225","Henry Joost, Ariel Schulman","Jamie Foxx, Joseph Gordon-Levitt, Dominique Fi...",When a pill that gives its users unpredictable...
1,The Tax Collector,(2020),4.7,"Action, Crime, Drama",95,"3,520",David Ayer,"Bobby Soto, Cinthya Carmona, Shia LaBeouf, Jos...","A ""tax collector"" working for a local crime lo..."
2,Perry Mason,(2020– ),7.6,"Crime, Drama",60,"8,051",Not Found,"Matthew Rhys, Juliet Rylance, Chris Chalk, She...","In booming 1932 Los Angeles, a down-and-out de..."
3,Yellowstone,(2018– ),8.5,"Drama, Western",60,"19,654",Not Found,"Kevin Costner, Luke Grimes, Kelly Reilly, Wes ...",A ranching family in Montana faces off against...
4,Dark,(2017–2020),8.8,"Crime, Drama, Mystery",60,"241,248",Not Found,"Louis Hofmann, Karoline Eichhorn, Lisa Vicari,...","A family saga with a supernatural twist, set i..."
...,...,...,...,...,...,...,...,...,...
60020,8 Seconds,(2015),6.5,"Biography, Drama, Fantasy",120,"1,769","Ömer Faruk Sorak, Birkan Pusa","Esra Inal, Ceylin Adiyaman, Gulay Baltaci, Bat...",ESRA leads two parallel lives. In her worldly ...
60021,Heart of the West,(1936),6.9,Western,63,132,Howard Bretherton,"William Boyd, James Ellison, George 'Gabby' Ha...",Ranch owner Sally Jordan is engaged in a fence...
60022,Matrubhoomi: A Nation Without Women,(2003),7.7,Drama,99,"1,944",Manish Jha,"Tulip Joshi, Sudhir Pandey, Sushant Singh, Pan...",A disturbing saga about male chauvinism and mi...
60023,Talash,(I) (2019),6.4,"Action, Adventure, Comedy",Not Found,32,Zeeshan Khan,"Saleem Mairaj, Mustafa Qureshi, Noaman Sami, A...",Add a Plot\n
